In [1]:
!pip install psycopg2-binary


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
%%markdown
## Prereq
Run the following in the DB
```
CREATE USER your_user WITH PASSWORD 'your_pw';

GRANT ALL PRIVILEGES ON DATABASE chat_db TO chat_user;

CREATE TABLE IF NOT EXISTS chat_messages (
    id VARCHAR(64) PRIMARY key,
    chat_messages JSONB NOT NULL
);
```

## Prereq
Run the following in the DB
```
CREATE USER chat_user WITH PASSWORD 'chatuser@1234';

GRANT ALL PRIVILEGES ON DATABASE chat_db TO chat_user;

CREATE TABLE IF NOT EXISTS chat_messages (
    id VARCHAR(64) PRIMARY key,
    chat_messages JSONB NOT NULL
);
```


In [3]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, SystemMessage, HumanMessage, messages_to_dict, BaseMessage, messages_from_dict, trim_messages
from typing import Sequence
from dotenv import load_dotenv
import uuid
import json
import psycopg2
from psycopg2.extras import Json

In [4]:
MODEL = "gpt-4o-mini"
DB_HOST = 'localhost'
DB_PORT = '5433'
DB_NAME = 'chat_db'
DB_USER = ''
DB_PASSWORD = ''

load_dotenv()

True

In [ ]:
model = ChatOpenAI(model=MODEL)

In [26]:
%%markdown
## DB Functions and connections

## DB Functions and connections


In [5]:
def insert_chat_message(chat_dict):
    try:
        # Generate a UUID4
        chat_id = uuid.uuid4()

        # Insert the data into the table
        insert_query = """
            INSERT INTO chat_messages (id, chat_messages)
            VALUES (%s, %s)
        """
        .execute(insert_query, (str(chat_id), Json(chat_dict)))

        # Commit the transaction
        connection.commit()

        print(f"Inserted chat message with UUID: {chat_id}")
    except Exception as e:
        print(f"Error inserting data: {e}")
        connection.rollback()

In [7]:
try:
    connection = psycopg2.connect(
        host=DB_HOST,
        port=DB_PORT,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD
    )
    cursor = connection.cursor()
    print("Connected to the PostgreSQL database successfully.")
except Exception as e:
    print(f"Error connecting to the database: {e}")
    exit(1)

Connected to the PostgreSQL database successfully.


In [25]:
%%markdown
## Improvement Over Last Version. Storing chat in a python list and then at the end of conversation in postgres.

## Improvement Over Last Version. Storing chat in a python list and then at the end of conversation in postgres.


In [22]:
messages: Sequence[BaseMessage] = []
system_prompt = SystemMessage(content="Your name is 'Agustrama' and you are a helpful assistant responsible for helping the users in the best possible way. On the first conversation make sure to introduce your self.")
messages.append(system_prompt)

In [24]:
while True:
    msg = input("User: ")
    if msg == 'q':
        end_statement = f"Agustrama: Thank you very much for contacting me. Hope to see you soon."
        msg_dict = messages_to_dict(messages)
        print(end_statement)
        insert_chat_message(msg_dict)
        break;
    
    user_prompt = HumanMessage(content=msg)
    messages.append(user_prompt)
    
    if len(messages) >= 7:
        chat_messages = trim_messages(
            messages,
            token_counter=len,
            max_tokens=5,
            start_on="human",
            end_on=("human", "tool"),
            include_system=True,
            allow_partial=False
        )
    
        model_response = model.invoke(chat_messages)
        print(f"Agustrama: {model_response.content}")
        messages.append(model_response)
    else:
        model_response = model.invoke(messages)
        print(f"Agustrama: {model_response.content}")
        messages.append(model_response)

User:  Hi


Agustrama: Hello! I'm Agustrama, your helpful assistant. How can I assist you today?


User:  Can you tell me what are some good budget resorts inside masai mara national park?


Agustrama: Sure! Here are some budget-friendly resorts and lodges in and around the Maasai Mara National Reserve that offer a good experience without breaking the bank:

1. **Mara River Lodge**: Located along the banks of the Mara River, this lodge provides a beautiful setting with comfortable accommodations and great wildlife viewing opportunities.

2. **Enchoro Wildlife Camp**: This is a tented camp that offers a more rustic experience while still providing essential amenities. It's located close to the park and is affordable.

3. **Mara Explorers Camp**: An eco-friendly camp that offers budget tents, meals, and guided safari options. It's known for its friendly atmosphere and proximity to wildlife.

4. **Olumara Tented Camp**: This camp offers budget options with comfortable tents and is located near the park's entrance. They also provide guided safari tours.

5. **Fig Tree Camp**: This camp offers a mix of luxury and budget accommodations, with options for camping or staying in ten

User:  q


Agustrama: Thank you very much for contacting me. Hope to see you soon.
Inserted chat message with UUID: ad9c0f15-6319-4f2c-8fbc-736aaf91d8d9
